In [1]:
import osmnx as ox
import networkx as nx
# import pandas as pd
# import numpy as np
from geopy.distance import great_circle
from heapq import heappop, heappush
from itertools import count
import matplotlib.pyplot as plt


from networkx.algorithms.shortest_paths.weighted import _weight_function

import sys
sys.path.append('../../../Multimodal_freight_USA/')

from mfreight.utils import plot
from mfreight.Multimodal.graph_utils import MultimodalNet

In [2]:
Net = MultimodalNet(path_u="../../../Multimodal_freight_USA/mfreight/Multimodal/data/multimodal_G_tot_u_w_price.plk")

# Astar

In [3]:
def heuristic_func_co2(u,v):
    G=Net.G_multimodal_u
    return great_circle((G.nodes[u]['y'],G.nodes[u]['x']),
                       (G.nodes[v]['y'],G.nodes[v]['x'])).miles * 0.01213 #* 0.05001, 0.01213


def astar_path(G, source, target, heuristic=None, weight="weight", capture_list=[None], save=False, show=False, res=150, title='Astar'):
    """Modified Astar. 
    Enter a capture_list to show the nodes explorations for the defined Dijkstra rank of the algorithm.
    """

    if source not in G or target not in G:
        msg = f"Either source {source} or target {target} is not in G"
        raise nx.NodeNotFound(msg)

    if heuristic is None:
        def heuristic(u, v):
            return 0

    push = heappush
    pop = heappop
    weight = _weight_function(G, weight)

    c = count()
    queue = [(0, next(c), source, 0, None)]

    enqueued = {}
    explored = {}
    
    counter = 0

    while queue:
        _, rank, curnode, dist, parent = pop(queue)
        counter += 1
        
        # Generate plots
        if counter in capture_list:
            print(f'Subgraph_nodes: {len(G.subgraph(explored.keys()))}')
            orig=G.nodes[source]['geometry']
            dest=G.nodes[target]['geometry']
            if save:
                plot.plot_multimodal_path_search(G.subgraph(explored.keys()),
                                                 orig=orig, dest=dest,
                                                 save_path=f'Images/Astar{counter}',
                                                 show=show,res=150, title=title)
            else:
                plot.plot_multimodal_path_search(G.subgraph(explored.keys()), orig=orig, dest=dest, save_path=None, show=show, title=title)

        if curnode == target:
            path = [curnode]
            node = parent
            while node is not None:
                path.append(node)
                node = explored[node]
            path.reverse()
            if capture_list:
                orig=G.nodes[source]['geometry']
                dest=G.nodes[target]['geometry']
                if save:
                    print('saved', len(explored.keys()), counter)
                    plot.plot_multimodal_path_search(G.subgraph(explored.keys()), orig=orig, dest=dest, save_path=f'Images/Astar{counter}',show=show,res=150, title=title)
                    remaining=[i for i in capture_list if i>counter]
                    for i in remaining:
                        print(i)       
                        plot.plot_multimodal_route(G, orig=orig, dest=dest, save_path=f'Images/Astar{i}', route=path, show=show,res=150, title=title)
                else:
                    plot.plot_multimodal_route(G, orig=orig, dest=dest, save_path=None, route=path, show=show, title=title)
            return path, explored, counter

        if curnode in explored:
            # Do not override the parent of starting node
            if explored[curnode] is None:
                continue

            # Skip bad paths that were enqueued before finding a better one
            qcost, h = enqueued[curnode]
            if qcost < dist:
                continue

        explored[curnode] = parent

        for neighbor, w in G[curnode].items():
            ncost = dist + weight(curnode, neighbor, w)
            if neighbor in enqueued:
                qcost, h = enqueued[neighbor]
                if qcost <= ncost:
                    continue
            else:
                h = heuristic(neighbor, target)
            enqueued[neighbor] = ncost, h
            push(queue, (ncost + h, next(c), neighbor, ncost, curnode))

    raise nx.NetworkXNoPath(f"Node {target} not reachable from {source}")

# Bidirectional Dijkstra

In [4]:
from networkx.algorithms.shortest_paths.weighted import _weight_function

def bidirectionnal_dijkstra(G, source, target, weight="weight", save=False, capture_list=None, show=False,res=150, title='Bi-Dijkstra'):

    if source not in G or target not in G:
        msg = f"Either source {source} or target {target} is not in G"
        raise nx.NodeNotFound(msg)

    if source == target:
        return (0, [source])

    weight = _weight_function(G, weight)
    push = heappush
    pop = heappop
    # Init:  [Forward, Backward]
    dists = [{}, {}]  # dictionary of final distances
    paths = [{source: [source]}, {target: [target]}]  # dictionary of paths
    fringe = [[], []]  # heap of (distance, node) for choosing node to expand
    seen = [{source: 0}, {target: 0}]  # dict of distances to seen nodes
    c = count()
    counter=0
    # initialize fringe heap
    push(fringe[0], (0, next(c), source))
    push(fringe[1], (0, next(c), target))
    # neighs for extracting correct neighbor information
    if G.is_directed():
        neighs = [G._succ, G._pred]
    else:
        neighs = [G._adj, G._adj]
    # variables to hold shortest discovered path
    # finaldist = 1e30000
    finalpath = []
    dir = 1
    while fringe[0] and fringe[1]:
        # choose direction
        # dir == 0 is forward direction and dir == 1 is back
        dir = 1 - dir
        # extract closest to expand
        (dist, rank, v) = pop(fringe[dir])
        counter+=1
        
        if counter in capture_list:
            explored = list(seen[0].keys())
            explored.extend(list(seen[1].keys()))
            print(f'Subgraph_nodes: {len(G.subgraph(explored))}')
            orig=G.nodes[source]['geometry']
            dest=G.nodes[target]['geometry']
            if save:
                plot.plot_multimodal_path_search(G.subgraph(explored), orig=orig, dest=dest, save_path=f'Images/BiDijkstra{counter}',show=show,res=150, title=title)
            else:
                plot.plot_multimodal_path_search(G.subgraph(explored), orig=orig, dest=dest, save_path=None, show=show, title=title)
                
                
        if v in dists[dir]:
            # Shortest path to v has already been found
            continue
        # update distance
        dists[dir][v] = dist  # equal to seen[dir][v]
        if v in dists[1 - dir]:
            # if we have scanned v in both directions we are done
            # we have now discovered the shortest path
            if capture_list:
                orig=G.nodes[source]['geometry']
                dest=G.nodes[target]['geometry']
                if save:
                    explored = list(seen[0].keys())
                    explored.extend(list(seen[1].keys()))
                    print(f'Subgraph_nodes_end: {len(G.subgraph(explored))}')
                    plot.plot_multimodal_path_search(G.subgraph(explored), orig=orig, dest=dest, save_path=f'Images/BiDijkstra{counter}',show=show,res=150, title=title)
                    remaining=[i for i in capture_list if i>counter]
                    for i in remaining:
                        print('path', i)
                        plot.plot_multimodal_route(G, orig=orig, dest=dest, save_path=f'Images/BiDijkstra{i}', route=path, show=show,res=150, title=title)
                else:
                    plot.plot_multimodal_route(G, orig=orig, dest=dest, save_path=None, route=path, show=show)
            return (finalpath, finaldist, rank)
        

        for w, d in neighs[dir][v].items():
            if dir == 0:  # forward
                vwLength = dists[dir][v] + weight(v, w, d)
            else:  # back, must remember to change v,w->w,v
                vwLength = dists[dir][v] + weight(w, v, d)
            if w in dists[dir]:
                if vwLength < dists[dir][w]:
                    raise ValueError("Contradictory paths found: negative weights?")
            elif w not in seen[dir] or vwLength < seen[dir][w]:
                # relaxing
                seen[dir][w] = vwLength
                push(fringe[dir], (vwLength, next(c), w))
                paths[dir][w] = paths[dir][v] + [w]
                if w in seen[0] and w in seen[1]:
                    # see if this path is better than than the already
                    # discovered shortest path
                    totaldist = seen[0][w] + seen[1][w]
                    if finalpath == [] or finaldist > totaldist:
                        finaldist = totaldist
                        revpath = paths[1][w][:]
                        revpath.reverse()
                        finalpath = paths[0][w] + revpath[1:]
    raise nx.NetworkXNoPath(f"No path between {source} and {target}.")

# Dijkstra

In [5]:
def dijkstra(
    G, source, weight_str, pred=None, paths=None, cutoff=None, target=None, save=False, capture_list=None, show=False,res=150,title='Dijkstra'
):

    G_succ = G._succ if G.is_directed() else G._adj

    weight = lambda u, v, data: data.get(weight_str, 1)
    counter=0
    
    if paths is None:
        paths={source:[source]}
        

    push = heappush
    pop = heappop
    dist = {}  # dictionary of final distances
    seen = {}
    # fringe is heapq with 3-tuples (distance,c,node)
    # use the count c to avoid comparing nodes (may not be able to)
    c = count()
    fringe = []

    if source not in G:
        raise nx.NodeNotFound(f"Source {source} not in G")
    seen[source] = 0
    push(fringe, (0, next(c), source))
    while fringe:
        (d, rank, v) = pop(fringe)
        counter+=1
        
        if counter in capture_list:
            orig=G.nodes[source]['geometry']
            dest=G.nodes[target]['geometry']
            explored = list(seen.keys())
            print(f'Subgraph_nodes: {len(G.subgraph(explored))}')
            
            if save:
                plot.plot_multimodal_path_search(G.subgraph(explored), orig=orig, dest=dest, save_path=f'Images/Dijkstra{counter}',show=show,res=150, title=title)
            else:
                plot.plot_multimodal_path_search(G.subgraph(explored), orig=orig, dest=dest, save_path=None, show=show,res=150, title=title)
                
        if v in dist:
            continue  # already searched this node.
        dist[v] = d
        if v == target:
            break
      
        for u, e in G_succ[v].items():
            cost = weight(v, u, e)
            if cost is None:
                continue
            vu_dist = dist[v] + cost
            if cutoff is not None:
                if vu_dist > cutoff:
                    continue
            if u in dist:
                u_dist = dist[u]
                if vu_dist < u_dist:
                    raise ValueError("Contradictory paths found:", "negative weights?")
                elif pred is not None and vu_dist == u_dist:
                    pred[u].append(v)
            elif u not in seen or vu_dist < seen[u]:
                seen[u] = vu_dist
                push(fringe, (vu_dist, next(c), u))
                if paths is not None:
                    paths[u] = paths[v] + [u]
                if pred is not None:
                    pred[u] = [v]
            elif vu_dist == seen[u]:
                if pred is not None:
                    pred[u].append(v)

    # The optional predecessor and path dictionaries can be accessed
    # by the caller via the pred and paths objects passed as arguments.
    
    if capture_list:
        orig=G.nodes[source]['geometry']
        dest=G.nodes[target]['geometry']
        if save:
            explored = list(seen.keys())
            print(f'Subgraph_nodes_end: {len(G.subgraph(explored))}')
            plot.plot_multimodal_path_search(G.subgraph(explored), orig=orig, dest=dest, save_path=f'Images/Dijkstra{counter}',show=show, title=title)
            remaining=[i for i in capture_list if i>counter]
            print(remaining)
            for i in remaining:
                print('final_rank',counter)
                print(i)
                plot.plot_multimodal_route(G, orig=orig, dest=dest, save_path=f'Images/Dijkstra{i}', route=paths[target], show=show, res=150, title=title)
        else:
            plot.plot_multimodal_route(G, orig=orig, dest=dest, save_path=None, route=paths[target], show=show, res=150, title=title)
    return paths[target], dist, rank

# Run simulations

In [6]:
orig = (35.122246, -80.819619) # Charlotte
dest = (39.499301, -104.677064) # Denver


node_orig = ox.get_nearest_node(Net.G_reachable_nodes, orig, method="haversine", return_dist=False)
node_dest = ox.get_nearest_node(Net.G_reachable_nodes, dest, method="haversine", return_dist=False)
capture_list=list(range(0,70900,700))#[0,500,2000,10000, 100000, 200000]#
capture_list[0]=2
# capture_list = [67000]
target="CO2_eq_kg"

In [7]:
path, explored, counter  = astar_path(Net.G_multimodal_u,
                                  node_orig, node_dest,
                                  heuristic=heuristic_func_co2,
                                  weight=target,
                                  capture_list=capture_list,
                            save=True,res=150) #range(100,30000,400)

Subgraph_nodes: 1
Subgraph_nodes: 627
Subgraph_nodes: 1282
Subgraph_nodes: 1930
Subgraph_nodes: 2580
Subgraph_nodes: 3230
Subgraph_nodes: 3873
Subgraph_nodes: 4505
Subgraph_nodes: 5131
Subgraph_nodes: 5776
Subgraph_nodes: 6395
Subgraph_nodes: 7017
Subgraph_nodes: 7662
Subgraph_nodes: 8289
Subgraph_nodes: 8928
Subgraph_nodes: 9564
Subgraph_nodes: 10175
Subgraph_nodes: 10808
Subgraph_nodes: 11451
Subgraph_nodes: 12074
Subgraph_nodes: 12715
Subgraph_nodes: 13357
Subgraph_nodes: 13982
Subgraph_nodes: 14605
Subgraph_nodes: 15224
Subgraph_nodes: 15808
Subgraph_nodes: 16398
Subgraph_nodes: 17004
saved 17090 18996
19600
20300
21000
21700
22400
23100
23800
24500
25200
25900
26600
27300
28000
28700
29400
30100
30800
31500
32200
32900
33600
34300
35000
35700
36400
37100
37800
38500
39200
39900
40600
41300
42000
42700
43400
44100
44800
45500
46200
46900
47600
48300
49000
49700
50400
51100
51800
52500
53200
53900
54600
55300
56000
56700
57400
58100
58800
59500
60200
60900
61600
62300
63000
63700
64

In [8]:
len(explored)

17090

In [9]:
out = dijkstra(Net.G_multimodal_u, source=node_orig, weight_str=target, target=node_dest, save=True, capture_list=capture_list,res=150)

Subgraph_nodes: 5
Subgraph_nodes: 693
Subgraph_nodes: 1359
Subgraph_nodes: 2027
Subgraph_nodes: 2726
Subgraph_nodes: 3405
Subgraph_nodes: 4061
Subgraph_nodes: 4733
Subgraph_nodes: 5359
Subgraph_nodes: 6067
Subgraph_nodes: 6725
Subgraph_nodes: 7348
Subgraph_nodes: 7962
Subgraph_nodes: 8581
Subgraph_nodes: 9279
Subgraph_nodes: 9949
Subgraph_nodes: 10613
Subgraph_nodes: 11271
Subgraph_nodes: 11949
Subgraph_nodes: 12562
Subgraph_nodes: 13222
Subgraph_nodes: 13880
Subgraph_nodes: 14487
Subgraph_nodes: 15160
Subgraph_nodes: 15775
Subgraph_nodes: 16381
Subgraph_nodes: 16986
Subgraph_nodes: 17606
Subgraph_nodes: 18203
Subgraph_nodes: 18840
Subgraph_nodes: 19467
Subgraph_nodes: 20101
Subgraph_nodes: 20715
Subgraph_nodes: 21324
Subgraph_nodes: 21939
Subgraph_nodes: 22525
Subgraph_nodes: 23152
Subgraph_nodes: 23746
Subgraph_nodes: 24395
Subgraph_nodes: 25039
Subgraph_nodes: 25628
Subgraph_nodes: 26231
Subgraph_nodes: 26836
Subgraph_nodes: 27424
Subgraph_nodes: 28032
Subgraph_nodes: 28628
Subgraph

In [10]:
out = bidirectionnal_dijkstra(Net.G_multimodal_u, source=node_orig, target=node_dest, weight=target, save=True, capture_list=capture_list,res=150)

Subgraph_nodes: 6
Subgraph_nodes: 682
Subgraph_nodes: 1329
Subgraph_nodes: 2005
Subgraph_nodes: 2651
Subgraph_nodes: 3361
Subgraph_nodes: 4019
Subgraph_nodes: 4694
Subgraph_nodes: 5384
Subgraph_nodes: 6060
Subgraph_nodes: 6712
Subgraph_nodes: 7398
Subgraph_nodes: 8056
Subgraph_nodes: 8718
Subgraph_nodes: 9382
Subgraph_nodes: 10011
Subgraph_nodes: 10674
Subgraph_nodes: 11372
Subgraph_nodes: 12092
Subgraph_nodes: 12765
Subgraph_nodes: 13397
Subgraph_nodes: 14024
Subgraph_nodes: 14685
Subgraph_nodes: 15288
Subgraph_nodes: 15927
Subgraph_nodes: 16513
Subgraph_nodes: 17180
Subgraph_nodes: 17846
Subgraph_nodes: 18523
Subgraph_nodes: 19184
Subgraph_nodes: 19898
Subgraph_nodes: 20580
Subgraph_nodes: 21240
Subgraph_nodes: 21916
Subgraph_nodes: 22528
Subgraph_nodes: 23145
Subgraph_nodes: 23787
Subgraph_nodes: 24428
Subgraph_nodes: 25046
Subgraph_nodes: 25684
Subgraph_nodes: 26370
Subgraph_nodes: 27033
Subgraph_nodes: 27716
Subgraph_nodes: 28339
Subgraph_nodes: 28947
Subgraph_nodes: 29603
Subgrap

In [11]:
departure=(35.122246, -80.819619)
arrival=(39.499301, -104.677064)

node_orig = ox.get_nearest_node(Net.G_reachable_nodes, departure, method="haversine", return_dist=False)
node_dest = ox.get_nearest_node(Net.G_reachable_nodes, arrival, method="haversine", return_dist=False)

In [16]:

path, explored, counter = astar_path(Net.G_multimodal_u,
                                  node_orig, node_dest,
                                  heuristic=heuristic_func_co2,
                                  weight=target,
                                  capture_list=[4],
                            save=True,res=150) #range(100,30000,400)

Subgraph_nodes: 3
saved 17090 18996


In [13]:
counter

18996

In [14]:
len(path)

248